In [18]:
## Package Upload
from __future__ import print_function
import sys
from pylsl import StreamInfo, StreamOutlet
import numpy as np
sys.path.append('..')  # help python find cyton.py relative to scripts folder
from openbci import wifi as bci
import logging

## Congifguration ##
logging.basicConfig(filename="test.log", format='%(asctime)s - %(levelname)s : %(message)s', level=logging.DEBUG)
logging.info('---------LOG START-------------')
# If you don't know your IP Address, you can use shield name option
# If you know IP, such as with wifi direct 192.168.4.1, then use ip_address='192.168.4.1'
shield_name = 'OpenBCI-296C'
shield_sr = 1000 # Sampling Rate

## Instantiate Wifi Shield Object ##

#shield = bci.OpenBCIWiFi(shield_name=shield_name, log=True, high_speed=True, sample_rate=shield_sr)

# For TP-Link Router Wifi Station
shield = bci.OpenBCIWiFi(ip_address='192.168.0.100',shield_name=shield_name, log=True, high_speed=True, sample_rate=shield_sr)

# For Wifi Direct
#shield = bci.OpenBCIWiFi(ip_address='192.168.4.1',shield_name=shield_name, log=True, high_speed=True, sample_rate=shield_sr, num_channels=16)

print("WiFi Shield Instantiated")

## Check Sampling Rate from WeDAQ ##
#print("Sample rate ",shield.getSampleRate())
#shield.set_sample_rate(shield_sr) # Makes sure to set sample rate


Welcome to OpenBCI Native WiFi Shield Driver - Please contribute code!
Opened socket on 192.168.0.103:53925
Init WiFi connection with IP: 192.168.0.100


RuntimeError: No board connected to WiFi Shield. To learn how to connect to a Cyton or Ganglion visit http://docs.openbci.com/Tutorials/03-Wifi_Getting_Started_Guide

In [ ]:
shield.wifi_write('A') # save data to SD card for 5 min (https://docs.openbci.com/docs/02Cyton/CytonSDK)

In [ ]:
shield.wifi_write('j') # stop logging data and close SD file

In [ ]:
shield.wifi_write('v') # get report of all register settings for the ADS1299 and the LIS3DH (Not working for some reason)

In [ ]:
shield.wifi_write('/1') # set debug mode on

In [ ]:
shield.wifi_write('d') # default setting for all channel

In [ ]:
print("Board Type:", shield.getBoardType())


# default channelnum
channelnum = 16

#shield.wifi_write('d') # set channel settings to default
#shield.wifi_write('?') # get report of all register settings for the ADS1299 and the LIS3DH

#shield.wifi_write('A')

## Define Board Type ## 
# isDaisy = True
# if isDaisy == False:
#     channelnum = 8
#     #shield.wifi_write('c')
#     shield.wifi_write('4');
#     print("Selecting only 8 channels")
# else:
#     shield.wifi_write('C')

print("Number of Channels:", shield.num_channels)

## Enable Synthetic Square Wave ##
#shield.test_signal(0); # 1 - enable syntetic square wave, 0 - disable syntethic square wave

In [ ]:
## Print to lsl

## For Cyton + Daisy
SCALE_FACTOR_EEG = (4500000)/24/(2**23-1) #uV/count
SCALE_FACTOR_AUX = 0.002 / (2**4) #G/count

print("Sample rate ",shield.getSampleRate())

# EEG Data Stream ##
print("Creating LSL stream for EEG. \nName: OpenBCIEEG1\nID: OpenBCItestEEG1\n")
info_eeg1 = StreamInfo('OpenBCIEEG1', 'EEG', 16, shield_sr, 'float32', 'OpenBCItestEEG1') # Define
outlet_eeg1 = StreamOutlet(info_eeg1)

## Acceleration Data Stream ##
#print("Creating LSL stream for ACCEL. \nName: OpenBCIACCEL1\nID: OpenBCItestACCEL1\n")
#info_accel1 = StreamInfo('OpenBCIACCEL1', 'ACCEL', 3, shield_sr, 'float32', 'OpenBCItestACCEL1')
#outlet_accel1 = StreamOutlet(info_accel1)

## define what Data to stream
def printData(sample):
    outlet_eeg1.push_sample(np.array(sample.channel_data)*SCALE_FACTOR_EEG)
    #outlet_accel1.push_sample(np.array(sample.accel_data)*SCALE_FACTOR_AUX)
    #outlet_accel1.push_sample(np.array(sample.sample_number))

shield.start_streaming(printData)
#shield.wifi_write('P')
shield.loop()

In [ ]:
shield.stop()

In [ ]:
shield.check_connection()